In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import naive_bayes, model_selection, metrics
import matplotlib.pyplot as plt

In [331]:
# This cell is for single view

data_set = pd.read_csv('D:/Datasets/Diabetes_Dataset.csv', header = None)
data_set.columns = ['Pregnant','Plasma_Glucose','DBP','Thickness','Insulin','BMI','Pedigree','Age','Class']
X = data_set.loc[:,data_set.columns != 'Class']
y = data_set['Class']


X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=.3, random_state=23)

gnb = naive_bayes.GaussianNB().fit(X_train,y_train)

y_pred = gnb.predict(X_test)



print('The accuracy of the single-view model trained using the Naive Bayes model is: ' + str(metrics.accuracy_score(y_test,y_pred)))




The accuracy of the single-view model trained using the Naive Bayes model is: 0.7575757575757576


In [19]:
X_train.shape

(92, 8)

In [20]:
def co_train(X_train,y_train,X_test,y_test):
    gnb = naive_bayes.GaussianNB().fit(X_train,y_train)
    y_pred2= gnb.predict(X_test) 

    y_prob = gnb.predict_proba(X_test)
                
    j = []

    for i in range(len(y_prob[:,1])):

        j.append(max(y_prob[i,0],y_prob[i,1]))

    print('Internal Classifier Accuracy: ' + str(metrics.accuracy_score(y_test,y_pred2)))


    X_first = np.column_stack((X_test,y_pred2,np.array(j),y_test))
    
    X_first = X_first[X_first[:,9].argsort()]
    n = round(.7 * len(X_first[:,9]))
    n_comp = len(X_first[:,9])- n

    total = np.row_stack((np.column_stack((sub_X_train2,sub_y_train2)),X_first[n:,0:9]))  #total new training
    total2 = np.row_stack((X_first[:n,0:9]))   #total new testing

    new_X = total[:,:-1]
    new_Y = total[:,-1:]

    test_X = total2[:,:-1]
    test_Y = total2[:,-1:]

    return [new_X, new_Y,test_X, test_Y,gnb]
#This initializes the first iteration of what we want

In [83]:
8#%% Co_training

# Train a very small portion
# Test. Testing that you are most confident about you add to the training data
# Retrain on that and continue to do this for a number of iterations. Use naive bayes

#def co_train(X,y):
    
X_train2, X_test2, y_train2, y_test2 = model_selection.train_test_split(X, y, test_size=0.2, random_state=21)

sub_X_train2, sub_X_test2, sub_y_train2, sub_y_test2 = model_selection.train_test_split(X_train2, y_train2, test_size=0.85, random_state=15)
  
    
j = co_train(sub_X_train2,sub_y_train2,sub_X_test2,sub_y_test2)
x = j[0]
y = j[1]
X = j[2]
Y = j[3]

for i in range(7):
    j = co_train(x,y,X,Y)
    x = j[0]
    y = j[1]
    X = j[2]
    Y = j[3]
    
model = j[4]

prediction = model.predict(X_test2)
print('\n\nThe accuracy of the multi-view model trained using the Naive Bayes model is: ' + str(metrics.accuracy_score(y_test2,prediction)))

    

Internal Classifier Accuracy: 0.6992337164750958
Internal Classifier Accuracy: 0.9013698630136986
Internal Classifier Accuracy: 0.9215686274509803
Internal Classifier Accuracy: 0.9044943820224719
Internal Classifier Accuracy: 0.728
Internal Classifier Accuracy: 0.8409090909090909
Internal Classifier Accuracy: 0.7903225806451613
Internal Classifier Accuracy: 0.5813953488372093


The accuracy of the multi-view model trained using the Naive Bayes model is: 0.7337662337662337


C:\Users\arman\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\arman\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\arman\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\arman\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

In [22]:
sub_X_train2.shape

(92, 8)

In [394]:
#real co-training
# This cell is for single view

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import naive_bayes, model_selection, metrics
import matplotlib.pyplot as plt


data_set = pd.read_csv('D:/Datasets/Diabetes_Dataset.csv', header = None)
data_set.columns = ['Pregnant','Plasma_Glucose','DBP','Thickness','Insulin','BMI','Pedigree','Age','Class']
X = data_set.loc[:,data_set.columns != 'Class']
y = data_set['Class']

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.98, random_state=15)


gnb = naive_bayes.GaussianNB().fit(X_train,y_train)

y_pred = gnb.predict(X_test)

print('The accuracy of the single-view model trained using the Naive Bayes model is: ' + str(metrics.accuracy_score(y_test,y_pred)))

print(X_train.shape)




The accuracy of the single-view model trained using the Naive Bayes model is: 0.5604249667994687
(15, 8)


In [407]:


L_X, U_X, L_y, U_y = model_selection.train_test_split(X, y, test_size=0.98, random_state=16)

L = np.column_stack((L_X,L_y))

U = U_X


V1_cols = ['Pregnant','Pedigree','Age','Plasma_Glucose']
V2_cols = ['DBP','BMI','Insulin','Thickness']

V1_train = L_X[V1_cols]
V2_train = L_X[V2_cols]

V1_test = U_X[V1_cols]
V2_test = U_X[V2_cols]

sub_y_train = L_y
n = 2

for i in range(35):
    h1 = naive_bayes.GaussianNB().fit(V1_train,sub_y_train)
    h2 = naive_bayes.GaussianNB().fit(V2_train,sub_y_train)

    y_pred1 = h1.predict(V1_test)
    y_pred2 = h2.predict(V2_test)

    indx = np.random.choice(len(U),replace = False,size = 20)
    indx_c = [x for x in range(len(U)) if x not in indx]


    U_prime = U.iloc[indx]

    U = U.iloc[indx_c]

    results = []

    predict1 = h1.predict(U_prime[V1_cols])
    proba1 = h1.predict_proba(U_prime[V1_cols])
    
    
    j1 = []
    for i in range(len(proba1[:,1])):
        j1.append(max(proba1[i,0],proba1[i,1]))
        
    j1 = np.array(j1)

    act_indx = (-j1).argsort()[:n]

    L = np.row_stack((L,np.column_stack((U_prime.iloc[act_indx],predict1[act_indx]))))

    
    predict2 = h2.predict(U_prime[V2_cols])
    proba2 = h2.predict_proba(U_prime[V2_cols])
    
    j2 = []

    for i in range(len(proba2[:,1])):
        j2.append(max(proba2[i,0],proba2[i,1]))        

    j2 = np.array(j2)
    
    act_indx = (-j2).argsort()[:n]

    L = np.row_stack((L,np.column_stack((U_prime.iloc[act_indx],predict2[act_indx]))))
    
    L = pd.DataFrame(L,columns= data_set.columns)
    V1_train = L[V1_cols]
    V2_train = L[V2_cols]
    
    sub_y_train = L['Class']
        
#print('The accuracy of the single-view model trained using the Naive Bayes model is: ' + str(metrics.accuracy_score(y_test,y_pred)))


In [408]:
gnb = naive_bayes.GaussianNB().fit(L.drop(['Class'],axis = 1),L['Class'])

y_pred = gnb.predict(U_X)

print('The accuracy of the single-view model trained using the Naive Bayes model is: ' + str(metrics.accuracy_score(U_y,y_pred)))




The accuracy of the single-view model trained using the Naive Bayes model is: 0.6440903054448871


In [290]:
print(U_X.shape)


(753, 8)
